In [2]:
import tensorflow as tf
import numpy as np

NUM_STATES = 10
NUM_ACTIONS = 2
GAMMA = 0.5


def hot_one_state(index):
    array = np.zeros(NUM_STATES)
    array[index] = 1.
    return array

# we will create a set of states, the agent get a reward for getting to the 5th one(4 in zero based array).
# the agent can go forward or backward by one state with wrapping(so if you go back from the 1st state you go to the end).
states = [(x == 4) for x in range(NUM_STATES)]
# [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]

session = tf.Session()
state = tf.placeholder("float", [None, NUM_STATES])
targets = tf.placeholder("float", [None, NUM_ACTIONS])

hidden_weights = tf.Variable(tf.constant(0., shape=[NUM_STATES, NUM_ACTIONS]))

output = tf.matmul(state, hidden_weights)

loss = tf.reduce_mean(tf.square(output - targets))
train_operation = tf.train.AdamOptimizer(0.1).minimize(loss)

session.run(tf.global_variables_initializer())

for i in range(50):
    state_batch = []
    rewards_batch = []

    # create a batch of states
    for state_index in range(NUM_STATES):
        state_batch.append(hot_one_state(state_index))

        minus_action_index = (state_index - 1) % NUM_STATES
        plus_action_index = (state_index + 1) % NUM_STATES

        minus_action_state_reward = session.run(output, feed_dict={state: [hot_one_state(minus_action_index)]})[0]
        #hghg = session.run(output, feed_dict={state: [hot_one_state(minus_action_index)]})        
        plus_action_state_reward = session.run(output, feed_dict={state: [hot_one_state(plus_action_index)]})

        # these action rewards are the results of the Q function for this state and the actions minus or plus
        action_rewards = [states[minus_action_index] + GAMMA * np.max(minus_action_state_reward),
                          states[plus_action_index] + GAMMA * np.max(plus_action_state_reward)]
        rewards_batch.append(action_rewards)
        #print(minus_action_state_reward, plus_action_state_reward)
        #print(np.max(minus_action_state_reward),np.max(plus_action_state_reward))

    session.run(train_operation, feed_dict={
        state: state_batch,
        targets: rewards_batch})

    print([states[x] + GAMMA *np.max(session.run(output, feed_dict={state: [hot_one_state(x)]}))
           for x in range(NUM_STATES)])

# The final output will look something like this, the values converage about the reward state.
# [0.16162321, 0.31524473, 0.62262321, 1.2479111, 1.6226232, 1.2479111, 0.62262321, 0.31

[0.0, 0.0, 0.0, 0.049999844282865524, 1.0, 0.049999844282865524, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.037204362452030182, 0.0997934490442276, 1.0372043624520302, 0.0997934490442276, 0.037204362452030182, 0.0, 0.0, 0.0]
[0.0, 0.031937744468450546, 0.077309519052505493, 0.14935240149497986, 1.0773095190525055, 0.14935240149497986, 0.077309519052505493, 0.031937744468450546, 0.0, 0.0]
[0.029053455218672752, 0.066414825618267059, 0.10729782283306122, 0.19860425591468811, 1.1072978228330612, 0.19860425591468811, 0.10729782283306122, 0.066414825618267059, 0.029053455218672752, 0.0]
[0.05968756228685379, 0.076092012226581573, 0.12421974539756775, 0.24740807712078094, 1.1242197453975677, 0.24740807712078094, 0.12421974539756775, 0.076092012226581573, 0.05968756228685379, 0.027271432802081108]
[0.054920297116041183, 0.069918297231197357, 0.13834135234355927, 0.2955460250377655, 1.1383413523435593, 0.2955460250377655, 0.13834135234355927, 0.069918297231197357, 0.054920297116041183, 0.054969124495983

[0.070857211947441101, 0.15087386965751648, 0.31431585550308228, 0.62715494632720947, 1.3143158555030823, 0.62715494632720947, 0.31431585550308228, 0.15087386965751648, 0.070857211947441101, 0.063372910022735596]
[0.069669447839260101, 0.15297435224056244, 0.31268343329429626, 0.62426161766052246, 1.3126834332942963, 0.62426161766052246, 0.31268343329429626, 0.15297435224056244, 0.069669447839260101, 0.05449787899851799]
[0.070217005908489227, 0.15642076730728149, 0.31071850657463074, 0.62258189916610718, 1.3107185065746307, 0.62258189916610718, 0.31071850657463074, 0.15642076730728149, 0.070217005908489227, 0.043185144662857056]
[0.072637028992176056, 0.15906375646591187, 0.3094353973865509, 0.62203550338745117, 1.3094353973865509, 0.62203550338745117, 0.3094353973865509, 0.15906375646591187, 0.072637028992176056, 0.031548619270324707]
[0.076507553458213806, 0.15943995118141174, 0.3096732497215271, 0.62252563238143921, 1.3096732497215271, 0.62252563238143921, 0.3096732497215271, 0.159

In [10]:
rewards_batch

[[0.02178521454334259, 0.15943995118141174],
 [0.076507553458213806, 0.3096732497215271],
 [0.15943995118141174, 0.62252563238143921],
 [0.3096732497215271, 1.3096732497215271],
 [0.62252563238143921, 0.62252563238143921],
 [1.3096732497215271, 0.3096732497215271],
 [0.62252563238143921, 0.15943995118141174],
 [0.3096732497215271, 0.076507553458213806],
 [0.15943995118141174, 0.02178521454334259],
 [0.076507553458213806, 0.076507553458213806]]

In [11]:
state_batch

[array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.]),
 array([ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.])]